<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Data Preprocessing in Python: Data Integration

Let's get some data first:

## 1. The CIA data:

In [77]:
import pandas as pd

link1="https://www.cia.gov/library/publications/resources/the-world-factbook/fields/274.html"

cia=pd.read_html(link1,header=0,attrs={'id': 'fieldListing'})[0]

### - Cleaning

In [78]:
# renaming both columns:
OldToNew={cia.columns[0]:'countries',cia.columns[1]:'co2'}
cia.rename(columns=OldToNew,inplace=True)

# cleaning Country (preventive)
cia.countries=cia.countries.str.strip()

# splitting second column, keeping main information. This overwrites co2.
result1=cia.co2.str.split(' Mt',expand=True)[0]
cia=cia.assign(co2=result1)

# extracting number and text from co2 into a dataframe, adding those to CIA
result2=cia.co2.str.extract('(?P<number>\d+\,*\.*\d*)\s*(?P<text>\D+)*')
cia=cia.assign(value=result2.number,
               unit=result2.text)

# deleting comma from newly created column'value'
cia.value=cia.value.str.replace(",","")

# recoding newly created column'unit':
replacements={'million': 10**6, "billion": 10**9,None:10**0}
cia.unit.replace(replacements,inplace=True)

# dropping unneeded column:
cia.drop(columns='co2',inplace=True)

### - Formatting

In [79]:
# Formatting numeric columns:

# value is text, now to numeric type:
cia=cia.assign(value=pd.to_numeric(cia.value))

# once value and unit are numeric, 
# new column 'co2_in_MT' is created from multiplying both.
cia=cia.assign(co2_in_MT=cia.value*cia.unit)

# value and unit were temporary columns, they can go now.
cia.drop(columns=['value','unit'],inplace=True)

In [80]:
cia.head()

,countries,co2_in_MT
0,Afghanistan,9067000.0
1,Albania,4500000.0
2,Algeria,135900000.0
3,American Samoa,361100.0
4,Angola,20950000.0


In [81]:
cia.dtypes

countries     object
co2_in_MT    float64
dtype: object

## 2. The DEMOCRACY INDEX data:

In [82]:
import pandas as pd
link2= "https://en.wikipedia.org/wiki/Democracy_Index" 

# getting the data frame in one step:
demodex=pd.read_html(link2,header=0,flavor='bs4',attrs={'class': 'wikitable sortable'})[0]

### - Cleaning

In [83]:
#bye row 167, and Rank
demodex=demodex.drop(index=167,columns=['Rank'])

# Simplifying column names to facilitate further work:
demodex.columns=demodex.columns.str.replace('\s+',"")

# preventive cleaning of numeric cell values:
NumericColNames=demodex.iloc[:,2:7].columns
badSymbols=[]
for columnName in NumericColNames:
    for cell in demodex[columnName]:
        try:
            float(cell)
        except:
            if cell not in badSymbols:
                badSymbols.append(cell)
            
import numpy as np  
demodex.loc[:,NumericColNames].replace(to_replace=badSymbols,value=np.nan,inplace=True)

# see if we have some strange value in the categorical columns:
demodex.iloc[:,-2::].apply(set).to_list()

[{'Authoritarian', 'Flawed democracy', 'Full democracy', 'Hybrid regime'},
 {'Africa',
  'Asia',
  'Europe',
  'Europe/Asia',
  'North America',
  'Oceania',
  'South America'}]

### - Formatting

In [84]:
# Formatting numeric columns

# save column names of the columns to change:
colsToChange=demodex.iloc[:,2:7].columns
# make changes NOT using iloc:
demodex[colsToChange]=demodex[colsToChange].apply(pd.to_numeric)


# Formatting categorical columns
# NOMINAL
demodex.Continent=pd.Categorical(demodex.Continent)
# ORDINAL
#rewrite the levels in order:
correctLevels=['Authoritarian', 'Hybrid regime', 'Flawed democracy','Full democracy']
demodex.Regimetype=pd.Categorical(demodex.Regimetype,categories=correctLevels,ordered=True)


In [85]:
demodex.dtypes

Country                           object
Score                             object
Electoralprocessandpluralism     float64
Functioningofgovernment          float64
Politicalparticipation           float64
Politicalculture                 float64
Civilliberties                   float64
Regimetype                      category
Continent                       category
dtype: object

In [86]:
demodex.Continent.cat.ordered

False

In [87]:
demodex.Regimetype.cat.ordered

True

## 3. The HUMAN DEVELOPMENT INDEX data:

* Go to this [website](http://hdr.undp.org/en/content/table-1-human-development-index-and-its-components-1) and download the CSV file about Human Develeopment Index and its components.
* Go to your gmail account and create a new GoogleSheet.
* Import the file into the GoogleSheet.
* Make sure commas do not appear in thousands.
* Create a CSV link to that data from Google.
* Clean and format the data

In [88]:
link1="https://docs.google.com/spreadsheets/d/e/2PACX-1vSNMpr1Dz3tvMqEAnmGAjaQi19zcsEiLZQn6f5ZM6ap4tzNy6bQUXtctNebCL4OyEN6UODUc0U7B9pf/pub?gid=1506000439&single=true&output=csv"

In [89]:
yourLink="https://docs.google.com/spreadsheets/d/e/2PACX-1vRZBSLXN-Uh_sOG-n25e8pGao1FCfLpNyWU4WtUD18fxxRzy5FnB-0PgRQYGcu7ZZCeLF8Y0lzA_2ql/pub?gid=1426880746&single=true&output=csv"

Collecting the data with the link from Google:

In [90]:
# need to call pandas:

import pandas as pd

hdi=pd.read_csv(yourLink)

### -  Cleaning

Keep only data for data frame

#### * Check first rows

In [91]:
# Is the header in the right place:
hdi.head(10)

,Unnamed: 0,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,SDG3,NaN,SDG4.3,NaN,SDG4.6,NaN,SDG8.5,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Human development index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
4,HDI rank,Country,(index value),NaN,(years),NaN,(years),NaN,(years),NaN,(2011 PPP $),NaN,NaN,NaN,NaN
5,NaN,NaN,2018,NaN,2018,NaN,2018,a,2018,a,2018.00,NaN,2018,NaN,2017
6,NaN,VERY HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,Norway,0.954,NaN,82.3,NaN,18.1,b,12.6,NaN,68058.62,NaN,5,NaN,1
8,2,Switzerland,0.946,NaN,83.6,NaN,16.2,NaN,13.4,NaN,59374.73,NaN,8,NaN,2
9,3,Ireland,0.942,NaN,82.1,NaN,18.8,b,12.5,c,55659.68,NaN,9,NaN,3


Two problems:

* The headers are in two different rows.
* The data starts in below zero.

Cleaning the top part:

* Saving the headers:

In [92]:
# for the first two columns:
hdi.iloc[4,:2].tolist()

['HDI rank', 'Country']

In [93]:
# for the rest of columns columns:
hdi.iloc[3,2:].tolist()

['Human development index (HDI) ',
 nan,
 'Life expectancy at birth',
 nan,
 'Expected years of schooling',
 nan,
 'Mean years of schooling',
 nan,
 'Gross national income (GNI) per capita',
 nan,
 'GNI per capita rank minus HDI rank',
 nan,
 'HDI rank']

In [94]:
# saving headers:
CurrentHeaders=hdi.iloc[4,:2].tolist()+hdi.iloc[3,2:].tolist()

# you saved this:
CurrentHeaders

['HDI rank',
 'Country',
 'Human development index (HDI) ',
 nan,
 'Life expectancy at birth',
 nan,
 'Expected years of schooling',
 nan,
 'Mean years of schooling',
 nan,
 'Gross national income (GNI) per capita',
 nan,
 'GNI per capita rank minus HDI rank',
 nan,
 'HDI rank']

* Setting the begining of data:

We need to get rid the top rows that are not part of the data frame:

In [95]:
# PREVIEW: deleting rows, NORWAY should be the first row:
hdi.drop(index=range(0,7)) #7 in range() will not be erased.

,Unnamed: 0,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
7,1,Norway,0.954,NaN,82.3,NaN,18.1,b,12.6,NaN,68058.62,NaN,5,NaN,1
8,2,Switzerland,0.946,NaN,83.6,NaN,16.2,NaN,13.4,NaN,59374.73,NaN,8,NaN,2
9,3,Ireland,0.942,NaN,82.1,NaN,18.8,b,12.5,c,55659.68,NaN,9,NaN,3
10,4,Germany,0.939,NaN,81.2,NaN,17.1,NaN,14.1,NaN,46945.95,NaN,15,NaN,4
11,4,"Hong Kong, China (SAR)",0.939,NaN,84.7,NaN,16.5,NaN,12.0,NaN,60220.80,NaN,5,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,NaN,Column 2: UNDESA (2019b).,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,NaN,Column 3: UNESCO Institute for Statistics (201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,NaN,Column 4: UNESCO Institute for Statistics (201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,"Column 5: World Bank (2019a), IMF (2019) and U...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Making effective the last code:

In [96]:
hdi.drop(index=range(0,7),inplace=True)

As soon as you delete rows, you should reset the indexes:

In [97]:
hdi.reset_index(drop=True,inplace=True)

You can rename the columns now:

In [98]:
# rename columns
hdi.columns=CurrentHeaders

So far:

In [99]:
hdi.head()

,HDI rank,Country,Human development index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
0,1,Norway,0.954,NaN,82.3,NaN,18.1,b,12.6,NaN,68058.62,NaN,5,NaN,1
1,2,Switzerland,0.946,NaN,83.6,NaN,16.2,NaN,13.4,NaN,59374.73,NaN,8,NaN,2
2,3,Ireland,0.942,NaN,82.1,NaN,18.8,b,12.5,c,55659.68,NaN,9,NaN,3
3,4,Germany,0.939,NaN,81.2,NaN,17.1,NaN,14.1,NaN,46945.95,NaN,15,NaN,4
4,4,"Hong Kong, China (SAR)",0.939,NaN,84.7,NaN,16.5,NaN,12.0,NaN,60220.80,NaN,5,NaN,6


#### * Check last rows

In [100]:
hdi.tail(65) #change until you see last data row

,HDI rank,Country,Human development index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
197,..,Somalia,..,NaN,57.1,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..
198,..,Tuvalu,..,NaN,..,NaN,12.3,NaN,..,NaN,5408.95,NaN,..,NaN,..
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,NaN,Human development groups,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,NaN,Very high human development,0.892,NaN,79.5,NaN,16.4,NaN,12.0,NaN,40111.57,NaN,—,NaN,—
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,NaN,Column 2: UNDESA (2019b).,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,NaN,Column 3: UNESCO Institute for Statistics (201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,NaN,Column 4: UNESCO Institute for Statistics (201...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,NaN,"Column 5: World Bank (2019a), IMF (2019) and U...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Based on what is diSplayed, you should keep data up to Tuvalu country:

In [101]:
# deleting: preview
hdi.drop(index=range(199,262)) # rows starting from 199 will be erased

,HDI rank,Country,Human development index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
0,1,Norway,0.954,NaN,82.3,NaN,18.1,b,12.6,NaN,68058.62,NaN,5,NaN,1
1,2,Switzerland,0.946,NaN,83.6,NaN,16.2,NaN,13.4,NaN,59374.73,NaN,8,NaN,2
2,3,Ireland,0.942,NaN,82.1,NaN,18.8,b,12.5,c,55659.68,NaN,9,NaN,3
3,4,Germany,0.939,NaN,81.2,NaN,17.1,NaN,14.1,NaN,46945.95,NaN,15,NaN,4
4,4,"Hong Kong, China (SAR)",0.939,NaN,84.7,NaN,16.5,NaN,12.0,NaN,60220.80,NaN,5,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,..,Monaco,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..
195,..,Nauru,..,NaN,..,NaN,11.3,e,..,NaN,17312.59,NaN,..,NaN,..
196,..,San Marino,..,NaN,..,NaN,15.1,NaN,..,NaN,..,NaN,..,NaN,..
197,..,Somalia,..,NaN,57.1,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..


Making effective the last code:

In [102]:
hdi.drop(index=range(199,262),inplace=True) # should I reset indexes?

So far:

In [103]:
hdi

,HDI rank,Country,Human development index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
0,1,Norway,0.954,NaN,82.3,NaN,18.1,b,12.6,NaN,68058.62,NaN,5,NaN,1
1,2,Switzerland,0.946,NaN,83.6,NaN,16.2,NaN,13.4,NaN,59374.73,NaN,8,NaN,2
2,3,Ireland,0.942,NaN,82.1,NaN,18.8,b,12.5,c,55659.68,NaN,9,NaN,3
3,4,Germany,0.939,NaN,81.2,NaN,17.1,NaN,14.1,NaN,46945.95,NaN,15,NaN,4
4,4,"Hong Kong, China (SAR)",0.939,NaN,84.7,NaN,16.5,NaN,12.0,NaN,60220.80,NaN,5,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,..,Monaco,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..
195,..,Nauru,..,NaN,..,NaN,11.3,e,..,NaN,17312.59,NaN,..,NaN,..
196,..,San Marino,..,NaN,..,NaN,15.1,NaN,..,NaN,..,NaN,..,NaN,..
197,..,Somalia,..,NaN,57.1,NaN,..,NaN,..,NaN,..,NaN,..,NaN,..


#### * Check the column names

You have columns with names "missing", you want to avoid those:

In [104]:
# Get good columns

GoodHeaders=[header for header in CurrentHeaders if str(header) != 'nan']
#
GoodHeaders

['HDI rank',
 'Country',
 'Human development index (HDI) ',
 'Life expectancy at birth',
 'Expected years of schooling',
 'Mean years of schooling',
 'Gross national income (GNI) per capita',
 'GNI per capita rank minus HDI rank',
 'HDI rank']

The column **HDI rank** appears twice, because it appears twice in the data. Also, you have another rank **GNI per capita rank minus HDI rank'**. Ranks are repetitive because we have the scores. So, you can always get rid of them in this and other similar situations:

In [105]:
FinalHeaders=[h for h in GoodHeaders if 'rank' not in h]
# then
FinalHeaders

['Country',
 'Human development index (HDI) ',
 'Life expectancy at birth',
 'Expected years of schooling',
 'Mean years of schooling',
 'Gross national income (GNI) per capita']

In [106]:
# remember loc works with names, not with positions:

hdi.loc[:,FinalHeaders]

,Country,Human development index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita
0,Norway,0.954,82.3,18.1,12.6,68058.62
1,Switzerland,0.946,83.6,16.2,13.4,59374.73
2,Ireland,0.942,82.1,18.8,12.5,55659.68
3,Germany,0.939,81.2,17.1,14.1,46945.95
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80
...,...,...,...,...,...,...
194,Monaco,..,..,..,..,..
195,Nauru,..,..,11.3,..,17312.59
196,San Marino,..,..,15.1,..,..
197,Somalia,..,57.1,..,..,..


Saving previous result:

In [107]:
hdi=hdi.loc[:,FinalHeaders]

In [108]:
# you have:
hdi.head()

,Country,Human development index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita
0,Norway,0.954,82.3,18.1,12.6,68058.62
1,Switzerland,0.946,83.6,16.2,13.4,59374.73
2,Ireland,0.942,82.1,18.8,12.5,55659.68
3,Germany,0.939,81.2,17.1,14.1,46945.95
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80


The remaining column names need to improve:

In [109]:
# replace with '' (empty) the "blanks":
hdi.columns.str.replace("\s","") 

Index(['Country', 'Humandevelopmentindex(HDI)', 'Lifeexpectancyatbirth',
       'Expectedyearsofschooling', 'Meanyearsofschooling',
       'Grossnationalincome(GNI)percapita'],
      dtype='object')

In [110]:
# replace with '' (empty) consecutive word characters in parenthesis:
hdi.columns.str.replace("\(\w+\)","")

Index(['Country', 'Human development index  ', 'Life expectancy at birth',
       'Expected years of schooling', 'Mean years of schooling',
       'Gross national income  per capita'],
      dtype='object')

In [111]:
#or all combines
hdi.columns.str.replace("\s+|\(\w+\)","")

Index(['Country', 'Humandevelopmentindex', 'Lifeexpectancyatbirth',
       'Expectedyearsofschooling', 'Meanyearsofschooling',
       'Grossnationalincomepercapita'],
      dtype='object')

Saving result:

In [112]:
hdi.columns=hdi.columns.str.replace("\s+|\(\w+\)","")

In [113]:
hdi.head()

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita
0,Norway,0.954,82.3,18.1,12.6,68058.62
1,Switzerland,0.946,83.6,16.2,13.4,59374.73
2,Ireland,0.942,82.1,18.8,12.5,55659.68
3,Germany,0.939,81.2,17.1,14.1,46945.95
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80


#### * Check the cell values

Find cells where all numeric data is missing, that is, a country with no data:

In [114]:
# check empty cells from second to last
#hdi.iloc[:,1:]
hdi[hdi.iloc[:,1:].isnull().all(axis=1)] #ALL

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita
62,HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN
117,MEDIUM HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN
155,LOW HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN
192,OTHER COUNTRIES OR TERRITORIES,NaN,NaN,NaN,NaN,NaN


These are no countries. They were subtitles for groups of countries. 

You need the opposite to that:

In [115]:
# the opposite
hdi[hdi.iloc[:,1:].notnull().all(axis=1)]

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita
0,Norway,0.954,82.3,18.1,12.6,68058.62
1,Switzerland,0.946,83.6,16.2,13.4,59374.73
2,Ireland,0.942,82.1,18.8,12.5,55659.68
3,Germany,0.939,81.2,17.1,14.1,46945.95
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80
...,...,...,...,...,...,...
194,Monaco,..,..,..,..,..
195,Nauru,..,..,11.3,..,17312.59
196,San Marino,..,..,15.1,..,..
197,Somalia,..,57.1,..,..,..


Keepin what you need:

In [116]:
hdi=hdi[hdi.iloc[:,1:].notnull().all(axis=1)]

The last code deleted rows, then indexes need to be reset:

In [117]:
hdi.reset_index(drop=True, inplace=True)

So far:

In [118]:
hdi

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita
0,Norway,0.954,82.3,18.1,12.6,68058.62
1,Switzerland,0.946,83.6,16.2,13.4,59374.73
2,Ireland,0.942,82.1,18.8,12.5,55659.68
3,Germany,0.939,81.2,17.1,14.1,46945.95
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80
...,...,...,...,...,...,...
190,Monaco,..,..,..,..,..
191,Nauru,..,..,11.3,..,17312.59
192,San Marino,..,..,15.1,..,..
193,Somalia,..,57.1,..,..,..


#### * Check the cell values

Above you see some characters that are not numbers, are they missing values:

In [119]:
hdi[hdi.iloc[:,1:].isnull().any(axis=1)] #ANY

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita


There are symbols that represent missing values, but they are not recognized so. We need to check what values in those **numeric** columns are not numbers:

In [120]:
badHDISymbols=[] # list for bad symbols

NumericColNames=hdi.iloc[:,1:].columns # save names of columns with numeric data

for columnName in NumericColNames:# visit every column name
    for cell in hdi[columnName]:# visit every cell for that column
        try:
            float(cell) # try this
        except: # if not possible:            
            if cell not in badHDISymbols:# if cell is not in the list                
                badHDISymbols.append(cell)# add it to the list

# you get:
badHDISymbols

['..']

You need to replace that value in the numeric columns:

In [121]:
import numpy as np

hdi.replace(to_replace=badHDISymbols,value=np.nan,inplace=True)

In [122]:
# you have:
hdi

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita
0,Norway,0.954,82.3,18.1,12.6,68058.62
1,Switzerland,0.946,83.6,16.2,13.4,59374.73
2,Ireland,0.942,82.1,18.8,12.5,55659.68
3,Germany,0.939,81.2,17.1,14.1,46945.95
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80
...,...,...,...,...,...,...
190,Monaco,NaN,NaN,NaN,NaN,NaN
191,Nauru,NaN,NaN,11.3,NaN,17312.59
192,San Marino,NaN,NaN,15.1,NaN,NaN
193,Somalia,NaN,57.1,NaN,NaN,NaN


When you have numbers in your columns, you can request statistical summaries:

In [123]:
hdi.describe(include='all')

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita
count,195,189,191,193,189,191
unique,195,153,132,93,89,191
top,Egypt,0.724,82.1,12.7,9.7,56140.23
freq,1,4,5,6,6,1


You are not getting those because they lack format.

### -  Formatting

Check the data types:

In [124]:
hdi.dtypes

Country                         object
Humandevelopmentindex           object
Lifeexpectancyatbirth           object
Expectedyearsofschooling        object
Meanyearsofschooling            object
Grossnationalincomepercapita    object
dtype: object

Country can remain as an object (text), but not the rest.

* **Formatting into numeric type**:

In [125]:
# as easy as:

hdi[NumericColNames]=hdi.loc[:,NumericColNames].apply(pd.to_numeric)

In [126]:
#recheck
hdi.dtypes

Country                          object
Humandevelopmentindex           float64
Lifeexpectancyatbirth           float64
Expectedyearsofschooling        float64
Meanyearsofschooling            float64
Grossnationalincomepercapita    float64
dtype: object

In [127]:
# recheck
hdi.describe(include='all')

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita
count,195,189.000000,191.000000,193.000000,189.000000,191.000000
unique,195,NaN,NaN,NaN,NaN,NaN
top,Egypt,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.713450,72.417277,13.212435,8.613228,18368.072461
std,NaN,0.150795,7.508442,2.933508,3.084619,19627.352304
min,NaN,0.377000,52.800000,5.000000,1.600000,659.730000
25%,NaN,0.596000,67.200000,11.300000,6.300000,4009.435000
50%,NaN,0.728000,73.800000,13.100000,9.000000,11610.910000
75%,NaN,0.830000,77.700000,15.200000,11.300000,26535.420000


Some more information:

In [128]:
hdi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 6 columns):
Country                         195 non-null object
Humandevelopmentindex           189 non-null float64
Lifeexpectancyatbirth           191 non-null float64
Expectedyearsofschooling        193 non-null float64
Meanyearsofschooling            189 non-null float64
Grossnationalincomepercapita    191 non-null float64
dtypes: float64(5), object(1)
memory usage: 9.3+ KB


# INTEGRATING

Let's check the columns we have for the last three data frames:

In [129]:
cia.columns

Index(['countries', 'co2_in_MT'], dtype='object')

In [130]:
demodex.columns

Index(['Country', 'Score', 'Electoralprocessandpluralism',
       'Functioningofgovernment', 'Politicalparticipation', 'Politicalculture',
       'Civilliberties', 'Regimetype', 'Continent'],
      dtype='object')

In [131]:
hdi.columns

Index(['Country', 'Humandevelopmentindex', 'Lifeexpectancyatbirth',
       'Expectedyearsofschooling', 'Meanyearsofschooling',
       'Grossnationalincomepercapita'],
      dtype='object')

Integrating data sets needs the following considerations:

* Merging is done on two data frames.
* You need a common column to be used in both data frames. The column names can be different.
* The merge can keep only the full coincidences, or also the values not matched, which will help you detect possible extra cleaning.
* Pandas differentiates calls the **left** from the **right** data frames. The left is the one that requires the merge.

Let's see the case where the common columns have the same name:

In [132]:
#hdi and demodex have a common column: Country
hdi.merge(demodex)

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita,Score,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Regimetype,Continent
0,Norway,0.954,82.3,18.1,12.6,68058.62,9.87,10.00,9.64,10.00,10.00,9.71,Full democracy,Europe
1,Switzerland,0.946,83.6,16.2,13.4,59374.73,9.03,9.58,9.29,7.78,9.38,9.12,Full democracy,Europe
2,Ireland,0.942,82.1,18.8,12.5,55659.68,9.24,10.00,7.86,8.33,10.00,10.00,Full democracy,Europe
3,Germany,0.939,81.2,17.1,14.1,46945.95,8.68,9.58,8.57,8.33,7.50,9.41,Full democracy,Europe
4,Australia,0.938,83.3,22.1,12.7,44097.02,9.09,10.00,8.93,7.78,8.75,10.00,Full democracy,Oceania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Mali,0.427,58.9,7.6,2.4,1965.39,4.92,6.42,3.07,3.89,5.63,5.59,Hybrid regime,Africa
143,Burundi,0.423,61.2,11.3,3.1,659.73,2.15,0.00,0.07,3.33,5.00,2.35,Authoritarian,Africa
144,Chad,0.401,54.0,7.5,2.4,1715.57,1.61,0.00,0.00,1.67,3.75,2.65,Authoritarian,Africa
145,Central African Republic,0.381,52.8,7.6,4.3,776.68,1.32,1.25,0.00,1.11,1.88,2.35,Authoritarian,Africa


Notice that this results shows the full matches by default, resulting in 147 countries. Have you lost countries?

In [133]:
len(hdi), len(demodex)

(195, 167)

You can keep the whole **hdi** rows:

In [134]:
# all hdi and the ones in common with demodex:
hdi.merge(demodex,how='left') 

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita,Score,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Regimetype,Continent
0,Norway,0.954,82.3,18.1,12.6,68058.62,9.87,10.00,9.64,10.00,10.00,9.71,Full democracy,Europe
1,Switzerland,0.946,83.6,16.2,13.4,59374.73,9.03,9.58,9.29,7.78,9.38,9.12,Full democracy,Europe
2,Ireland,0.942,82.1,18.8,12.5,55659.68,9.24,10.00,7.86,8.33,10.00,10.00,Full democracy,Europe
3,Germany,0.939,81.2,17.1,14.1,46945.95,8.68,9.58,8.57,8.33,7.50,9.41,Full democracy,Europe
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Monaco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,Nauru,NaN,NaN,11.3,NaN,17312.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,San Marino,NaN,NaN,15.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,Somalia,NaN,57.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The merging method **outer** will give you more informative results:

In [135]:
hdi.merge(demodex,how='outer',indicator=True) # see last column

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita,Score,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Regimetype,Continent,_merge
0,Norway,0.954,82.3,18.1,12.6,68058.62,9.87,10.00,9.64,10.00,10.00,9.71,Full democracy,Europe,both
1,Switzerland,0.946,83.6,16.2,13.4,59374.73,9.03,9.58,9.29,7.78,9.38,9.12,Full democracy,Europe,both
2,Ireland,0.942,82.1,18.8,12.5,55659.68,9.24,10.00,7.86,8.33,10.00,10.00,Full democracy,Europe,both
3,Germany,0.939,81.2,17.1,14.1,46945.95,8.68,9.58,8.57,8.33,7.50,9.41,Full democracy,Europe,both
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Iran,NaN,NaN,NaN,NaN,NaN,2.38,0.00,2.86,4.44,3.13,1.47,Authoritarian,Asia,right_only
211,Laos,NaN,NaN,NaN,NaN,NaN,2.14,0.00,2.86,1.67,5.00,1.18,Authoritarian,Asia,right_only
212,Syria,NaN,NaN,NaN,NaN,NaN,1.43,0.00,0.00,2.78,4.38,0.00,Authoritarian,Asia,right_only
213,Democratic Republic of the Congo,NaN,NaN,NaN,NaN,NaN,1.13,0.00,0.00,1.67,3.13,0.88,Authoritarian,Africa,right_only


Let me save this result:

In [136]:
dirtyMerge1=hdi.merge(demodex,how='outer',indicator=True) 

Now, let me request the countries where the _demodex_ data frame found no match:

In [137]:
dirtyMerge1.loc[dirtyMerge1['_merge']=='right_only',"Country"]

195                    South Korea[n 1]
196                          Cape Verde
197                              Taiwan
198                      Czech Republic
199                           Hong Kong
200                             Moldova
201                            Tanzania
202                             Bolivia
203                         Ivory Coast
204                           Palestine
205                            Eswatini
206               Republic of the Congo
207                              Russia
208                             Vietnam
209                           Venezuela
210                                Iran
211                                Laos
212                               Syria
213    Democratic Republic of the Congo
214                         North Korea
Name: Country, dtype: object

Similarly, let me request the countries where the _hdi_ data frame found no match:

In [138]:
dirtyMerge1.loc[dirtyMerge1['_merge']=='left_only',"Country"]

4                   Hong Kong, China (SAR)
17                           Liechtenstein
22                     Korea (Republic of)
25                                 Czechia
35                                 Andorra
42                       Brunei Darussalam
48                      Russian Federation
54                                   Palau
55                                Barbados
59                                 Bahamas
61                              Seychelles
64              Iran (Islamic Republic of)
72                   Saint Kitts and Nevis
73                     Antigua and Barbuda
77                                 Grenada
89                             Saint Lucia
94        Saint Vincent and the Grenadines
96      Venezuela (Bolivarian Republic of)
97                                Dominica
102                                 Belize
103                               Maldives
104                                  Tonga
106                  Moldova (Republic of)
111        

At this point your data was clean, but yo discover there is still some pending work to do. If you decide to alter the names, you can fo this:

In [139]:
#dictionary of replacements:
replacements1={'South Korea[n 1]': 'Korea (Republic of)', 
              'Cape Verde':'Cabo Verde',
              'Czech Republic':'Czechia',
              'Hong Kong':'Hong Kong, China (SAR)',
              'Moldova':'Moldova (Republic of)',
              'Bolivia':'Bolivia (Plurinational State of)',
              'Tanzania':'Tanzania (United Republic of)',
              'Palestine':'Palestine, State of',
              'Ivory Coast':"Côte d'Ivoire",
              'Republic of the Congo':'Congo',
              'Venezuela':'Venezuela (Bolivarian Republic of)',
              'Vietnam':'Viet Nam',
              'Eswatini':'Eswatini (Kingdom of)',              
              'Russia':'Russian Federation',
              'Iran':'Iran (Islamic Republic of)',
              'Laos':"Lao People's Democratic Republic",
              'Democratic Republic of the Congo':'Congo (Democratic Republic of the)',
              'Syria':'Syrian Arab Republic',
              'North Korea': "Korea (Democratic People's Rep. of)" #check ""
             }

# replacing
demodex.Country.replace(replacements1,inplace=True)

You can do your default merge now, and you will get more matches:

In [140]:
hdi.merge(demodex)

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita,Score,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Regimetype,Continent
0,Norway,0.954,82.3,18.1,12.6,68058.62,9.87,10.00,9.64,10.00,10.00,9.71,Full democracy,Europe
1,Switzerland,0.946,83.6,16.2,13.4,59374.73,9.03,9.58,9.29,7.78,9.38,9.12,Full democracy,Europe
2,Ireland,0.942,82.1,18.8,12.5,55659.68,9.24,10.00,7.86,8.33,10.00,10.00,Full democracy,Europe
3,Germany,0.939,81.2,17.1,14.1,46945.95,8.68,9.58,8.57,8.33,7.50,9.41,Full democracy,Europe
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80,6.02,3.58,4.36,6.11,7.50,8.53,Flawed democracy,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,Burundi,0.423,61.2,11.3,3.1,659.73,2.15,0.00,0.07,3.33,5.00,2.35,Authoritarian,Africa
162,Chad,0.401,54.0,7.5,2.4,1715.57,1.61,0.00,0.00,1.67,3.75,2.65,Authoritarian,Africa
163,Central African Republic,0.381,52.8,7.6,4.3,776.68,1.32,1.25,0.00,1.11,1.88,2.35,Authoritarian,Africa
164,Niger,0.377,62.0,6.5,2.0,912.04,3.29,2.92,1.14,3.33,4.38,4.71,Authoritarian,Africa


Let's save this last result:

In [141]:
hdidemo=hdi.merge(demodex)

Now, let's merge this result with the other one whose common column has a different name:

In [142]:
hdidemo.merge(cia,left_on="Country", right_on='countries')

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita,Score,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Regimetype,Continent,countries,co2_in_MT
0,Norway,0.954,82.3,18.1,12.6,68058.62,9.87,10.00,9.64,10.00,10.00,9.71,Full democracy,Europe,Norway,39800000.0
1,Switzerland,0.946,83.6,16.2,13.4,59374.73,9.03,9.58,9.29,7.78,9.38,9.12,Full democracy,Europe,Switzerland,38950000.0
2,Ireland,0.942,82.1,18.8,12.5,55659.68,9.24,10.00,7.86,8.33,10.00,10.00,Full democracy,Europe,Ireland,36910000.0
3,Germany,0.939,81.2,17.1,14.1,46945.95,8.68,9.58,8.57,8.33,7.50,9.41,Full democracy,Europe,Germany,847600000.0
4,Australia,0.938,83.3,22.1,12.7,44097.02,9.09,10.00,8.93,7.78,8.75,10.00,Full democracy,Oceania,Australia,439100000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Mali,0.427,58.9,7.6,2.4,1965.39,4.92,6.42,3.07,3.89,5.63,5.59,Hybrid regime,Africa,Mali,3388000.0
143,Burundi,0.423,61.2,11.3,3.1,659.73,2.15,0.00,0.07,3.33,5.00,2.35,Authoritarian,Africa,Burundi,217000.0
144,Chad,0.401,54.0,7.5,2.4,1715.57,1.61,0.00,0.00,1.67,3.75,2.65,Authoritarian,Africa,Chad,342200.0
145,Central African Republic,0.381,52.8,7.6,4.3,776.68,1.32,1.25,0.00,1.11,1.88,2.35,Authoritarian,Africa,Central African Republic,413800.0


We are losing countries again. Let's use the **outer** method.

In [143]:
dirtyMerge2=hdidemo.merge(cia,left_on="Country", right_on='countries',how='outer',indicator=True) 

In [144]:
dirtyMerge2

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita,Score,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Regimetype,Continent,countries,co2_in_MT,_merge
0,Norway,0.954,82.3,18.1,12.6,68058.62,9.87,10.00,9.64,10.00,10.00,9.71,Full democracy,Europe,Norway,3.980000e+07,both
1,Switzerland,0.946,83.6,16.2,13.4,59374.73,9.03,9.58,9.29,7.78,9.38,9.12,Full democracy,Europe,Switzerland,3.895000e+07,both
2,Ireland,0.942,82.1,18.8,12.5,55659.68,9.24,10.00,7.86,8.33,10.00,10.00,Full democracy,Europe,Ireland,3.691000e+07,both
3,Germany,0.939,81.2,17.1,14.1,46945.95,8.68,9.58,8.57,8.33,7.50,9.41,Full democracy,Europe,Germany,8.476000e+08,both
4,"Hong Kong, China (SAR)",0.939,84.7,16.5,12.0,60220.80,6.02,3.58,4.36,6.11,7.50,8.53,Flawed democracy,Asia,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vietnam,2.353000e+08,right_only
231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Virgin Islands,2.764000e+06,right_only
232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Bank,3.113000e+06,right_only
233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Sahara,2.684000e+05,right_only


In [145]:
dirtyMerge2.loc[dirtyMerge2['_merge']=='left_only',"Country"].to_list()

['Hong Kong, China (SAR)',
 'Korea (Republic of)',
 'Russian Federation',
 'Iran (Islamic Republic of)',
 'Venezuela (Bolivarian Republic of)',
 'Moldova (Republic of)',
 'Bolivia (Plurinational State of)',
 'Viet Nam',
 'Palestine, State of',
 'Congo',
 'Eswatini (Kingdom of)',
 "Lao People's Democratic Republic",
 'Myanmar',
 'Syrian Arab Republic',
 'Tanzania (United Republic of)',
 "Côte d'Ivoire",
 'Gambia',
 'Congo (Democratic Republic of the)',
 "Korea (Democratic People's Rep. of)"]

In [146]:
dirtyMerge2.loc[dirtyMerge2['_merge']=='right_only',"countries"].to_list()

['American Samoa',
 'Antigua and Barbuda',
 'Aruba',
 'Bahamas, The',
 'Barbados',
 'Belize',
 'Bermuda',
 'Bolivia',
 'British Virgin Islands',
 'Brunei',
 'Burma',
 'Cayman Islands',
 'Congo, Democratic Republic of the',
 'Congo, Republic of the',
 'Cook Islands',
 "Cote d'Ivoire",
 'Dominica',
 'Eswatini',
 'European Union',
 'Falkland Islands (Islas Malvinas)',
 'Faroe Islands',
 'French Polynesia',
 'Gambia, The',
 'Gibraltar',
 'Greenland',
 'Grenada',
 'Guam',
 'Hong Kong',
 'Iran',
 'Jersey',
 'Kiribati',
 'Korea, North',
 'Korea, South',
 'Kosovo',
 'Laos',
 'Macau',
 'Maldives',
 'Marshall Islands',
 'Micronesia, Federated States of',
 'Moldova',
 'Montserrat',
 'Nauru',
 'New Caledonia',
 'Niue',
 'Puerto Rico',
 'Russia',
 'Saint Helena, Ascension, and Tristan da Cunha',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Pierre and Miquelon',
 'Saint Vincent and the Grenadines',
 'Samoa',
 'Sao Tome and Principe',
 'Seychelles',
 'Solomon Islands',
 'Somalia',
 'South Sudan'

In [147]:
replacements2={'Hong Kong, China (SAR)':'Hong Kong',
 'Korea (Republic of)':'Korea, South',
 'Russian Federation':'Russia',
 'Iran (Islamic Republic of)':'Iran',
 'Venezuela (Bolivarian Republic of)':'Venezuela',
 'Moldova (Republic of)':'Moldova',
 'Bolivia (Plurinational State of)':'Bolivia',
 'Viet Nam':'Vietnam',
 'Congo':'Congo, Republic of the',
 'Eswatini (Kingdom of)':'Eswatini',
 "Lao People's Democratic Republic":'Laos',
 'Myanmar':'Burma',
 'Syrian Arab Republic':'Syria',
 'Tanzania (United Republic of)':'Tanzania',
 "Côte d'Ivoire":"Cote d'Ivoire",
 'Gambia':'Gambia, The',
 'Congo (Democratic Republic of the)':'Congo, Democratic Republic of the'
}
 
 
 # replacing
hdidemo.Country.replace(replacements2,inplace=True)

Then:

In [148]:
hdidemo.merge(cia,left_on="Country", right_on='countries')

,Country,Humandevelopmentindex,Lifeexpectancyatbirth,Expectedyearsofschooling,Meanyearsofschooling,Grossnationalincomepercapita,Score,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Regimetype,Continent,countries,co2_in_MT
0,Norway,0.954,82.3,18.1,12.6,68058.62,9.87,10.00,9.64,10.00,10.00,9.71,Full democracy,Europe,Norway,39800000.0
1,Switzerland,0.946,83.6,16.2,13.4,59374.73,9.03,9.58,9.29,7.78,9.38,9.12,Full democracy,Europe,Switzerland,38950000.0
2,Ireland,0.942,82.1,18.8,12.5,55659.68,9.24,10.00,7.86,8.33,10.00,10.00,Full democracy,Europe,Ireland,36910000.0
3,Germany,0.939,81.2,17.1,14.1,46945.95,8.68,9.58,8.57,8.33,7.50,9.41,Full democracy,Europe,Germany,847600000.0
4,Hong Kong,0.939,84.7,16.5,12.0,60220.80,6.02,3.58,4.36,6.11,7.50,8.53,Flawed democracy,Asia,Hong Kong,102500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,Mali,0.427,58.9,7.6,2.4,1965.39,4.92,6.42,3.07,3.89,5.63,5.59,Hybrid regime,Africa,Mali,3388000.0
160,Burundi,0.423,61.2,11.3,3.1,659.73,2.15,0.00,0.07,3.33,5.00,2.35,Authoritarian,Africa,Burundi,217000.0
161,Chad,0.401,54.0,7.5,2.4,1715.57,1.61,0.00,0.00,1.67,3.75,2.65,Authoritarian,Africa,Chad,342200.0
162,Central African Republic,0.381,52.8,7.6,4.3,776.68,1.32,1.25,0.00,1.11,1.88,2.35,Authoritarian,Africa,Central African Republic,413800.0


In [149]:
hdidemocia=hdidemo.merge(cia,left_on="Country", right_on='countries')

In [150]:
hdidemocia.dtypes

Country                           object
Humandevelopmentindex            float64
Lifeexpectancyatbirth            float64
Expectedyearsofschooling         float64
Meanyearsofschooling             float64
Grossnationalincomepercapita     float64
Score                             object
Electoralprocessandpluralism     float64
Functioningofgovernment          float64
Politicalparticipation           float64
Politicalculture                 float64
Civilliberties                   float64
Regimetype                      category
Continent                       category
countries                         object
co2_in_MT                        float64
dtype: object

____
____


### <font color="red">Saving File to Disk</font>

#### For future use in Python:

In [151]:
hdidemocia.to_pickle("hdidemocia.pkl")
# you will need: DF=pd.read_pickle("hdidemocia.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://..../hdidemocia.pkl"),compression=None)

#### For future  use in R:

In [156]:
# this makes the data converted to RDS

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(hdidemocia,file="hdidemocia.RDS")

#In R, you call it with: DF = readRDS("hdidemocia.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://..../hdidemocia.RDS")

ModuleNotFoundError: No module named 'tzlocal'

In [157]:
# get the file to your computer:
# if it doesn't work, try this: https://stackoverflow.com/questions/53581023/google-colab-file-download-failed-to-fetch-error
from google.colab import files

files.download('hdidem.RDS')

ModuleNotFoundError: No module named 'google.colab'

## Exercise 11

1. Reload the original data from HDI.
2. Create a new data frame, with ONLY the name of countries and a dichotomical value that has a 1 if above the median HDI, and 0 otherwise.
3. Merge this new data frame with the **hdidemocia** data frame. Name the final output **hdidemocia2**
4. Save **hdidemocia2** for future use in Python and in R.

## Exercise 12

1. Clean and format your data set, the one you presented last week.
2. Save the result for future use in Python and in R.